In [17]:
import pandas as pd
import random as rd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [18]:
rd.seed(6)

In [20]:
skip = sorted(rd.sample(xrange(1, 2197291), 2187291))

In [21]:
train = pd.read_csv("act_train.csv", skiprows=skip)

In [22]:
train.columns.values

array(['people_id', 'activity_id', 'date', 'activity_category', 'char_1',
       'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7',
       'char_8', 'char_9', 'char_10', 'outcome'], dtype=object)

In [23]:
length = len(train)

train.apply(lambda v: sum(v.isnull())*1.0/length)

people_id            0.0000
activity_id          0.0000
date                 0.0000
activity_category    0.0000
char_1               0.9252
char_2               0.9252
char_3               0.9252
char_4               0.9252
char_5               0.9252
char_6               0.9252
char_7               0.9252
char_8               0.9252
char_9               0.9252
char_10              0.0748
outcome              0.0000
dtype: float64

In [24]:
people = pd.read_csv("people.csv")

people.columns.values

array(['people_id', 'char_1', 'group_1', 'char_2', 'date', 'char_3',
       'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9',
       'char_10', 'char_11', 'char_12', 'char_13', 'char_14', 'char_15',
       'char_16', 'char_17', 'char_18', 'char_19', 'char_20', 'char_21',
       'char_22', 'char_23', 'char_24', 'char_25', 'char_26', 'char_27',
       'char_28', 'char_29', 'char_30', 'char_31', 'char_32', 'char_33',
       'char_34', 'char_35', 'char_36', 'char_37', 'char_38'], dtype=object)

In [25]:
people.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [26]:
train_merged = pd.merge(left=people, right=train, on="people_id", suffixes=["_p", "_act"])

train_merged.columns.values

array(['people_id', 'char_1_p', 'group_1', 'char_2_p', 'date_p',
       'char_3_p', 'char_4_p', 'char_5_p', 'char_6_p', 'char_7_p',
       'char_8_p', 'char_9_p', 'char_10_p', 'char_11', 'char_12',
       'char_13', 'char_14', 'char_15', 'char_16', 'char_17', 'char_18',
       'char_19', 'char_20', 'char_21', 'char_22', 'char_23', 'char_24',
       'char_25', 'char_26', 'char_27', 'char_28', 'char_29', 'char_30',
       'char_31', 'char_32', 'char_33', 'char_34', 'char_35', 'char_36',
       'char_37', 'char_38', 'activity_id', 'date_act',
       'activity_category', 'char_1_act', 'char_2_act', 'char_3_act',
       'char_4_act', 'char_5_act', 'char_6_act', 'char_7_act',
       'char_8_act', 'char_9_act', 'char_10_act', 'outcome'], dtype=object)

In [32]:
train_merged_na = train_merged.apply(lambda v: sum(v.isnull())*1.0/length)
train_merged_na[train_merged_na>0]

char_1_act     0.9252
char_2_act     0.9252
char_3_act     0.9252
char_4_act     0.9252
char_5_act     0.9252
char_6_act     0.9252
char_7_act     0.9252
char_8_act     0.9252
char_9_act     0.9252
char_10_act    0.0748
dtype: float64

In [34]:
train_merged.char_1_act.value_counts()

type 2     181
type 5     159
type 1      74
type 12     71
type 3      67
type 7      29
type 10     22
type 6      17
type 26     15
type 17     15
type 23     13
type 13     10
type 11     10
type 16      9
type 8       8
type 15      7
type 29      7
type 4       5
type 9       4
type 14      4
type 30      4
type 36      3
type 25      2
type 19      2
type 20      2
type 24      2
type 41      2
type 28      1
type 40      1
type 27      1
type 44      1
Name: char_1_act, dtype: int64

In [44]:
train_merged.iloc[:,0:3].head()

,people_id,char_1_p,group_1
0,ppl_100035,type 2,group 9439
1,ppl_100087,type 2,group 17304
2,ppl_100146,type 2,group 7256
3,ppl_100192,type 2,group 11102
4,ppl_100221,type 2,group 14626


In [58]:
(train_merged.outcome == 0).sum()/10000.0

0.55689999999999995

In [60]:
from sklearn.linear_model import LogisticRegression
from patsy import dmatrices

In [67]:
Y, X = dmatrices("outcome~char_38", data=train_merged, return_type="dataframe")
y = Y["outcome"]

In [69]:
X[:10]

,Intercept,char_38
0,1.0,100.0
1,1.0,44.0
2,1.0,100.0
3,1.0,85.0
4,1.0,69.0
5,1.0,91.0
6,1.0,70.0
7,1.0,98.0
8,1.0,65.0
9,1.0,52.0


In [70]:
model = LogisticRegression()
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)